In [1]:
from pyspark.sql import SparkSession
import os

# os.environ['SPARK_HOME'] = "/Users/rutvik/Study/DBMS/spark-3.4.2-bin-hadoop3/"

In [2]:
spark = SparkSession.builder.appName("pyspark from public s3")\
       .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0")\
       .config("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider") \
       .getOrCreate()
sqlContext = SparkSession(spark)

24/04/12 16:10:47 WARN Utils: Your hostname, Rutvik.local resolves to a loopback address: 127.0.0.1; using 10.0.0.46 instead (on interface en0)
24/04/12 16:10:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/rutvik/.ivy2/cache
The jars for the packages stored in: /Users/rutvik/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-acd5b9af-9b86-4b84-8c06-9230bf5e014e;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 98ms :: artifacts dl 2ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-acd5b9af-9b86-4b84-8c06-9230bf5e014e
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/2ms)
24/04/12 16:10:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your plat

In [3]:
df = spark.read.format("csv").option("header","true").load("s3a://multi-token-completion/human_annotated/labeled_completions.csv")

24/04/12 16:10:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
df.createOrReplaceTempView("public_s3_bucket")

In [5]:
spark.sql("select * from public_s3_bucket").show(10)

+---+--------------------+-------------------+--------------------+--------+-------+----------------+--------------------+-------+
|_c0|         masked_sent|    original_phrase|suggested_completion|is_valid|is_fact|is_more_abstract|is_factually_correct| method|
+---+--------------------+-------------------+--------------------+--------+-------+----------------+--------------------+-------+
|  0|Line B was intend...|the State of Mexico|        buenos aires|       1|      1|             0.0|                 0.0|  t5-3b|
|  1|Line B was intend...|the State of Mexico|      Rio de Janeiro|       1|      1|             0.0|                 0.0|    ilm|
|  2|Line B was intend...|the State of Mexico|              Cuenca|       1|      1|             0.0|                 0.0|   EMAT|
|  3|Line B was intend...|the State of Mexico|            the u.s.|       1|      1|             1.0|                 0.0|t5-base|
|  4|Lloyd Richards, s...|             Raisin|                 day|       1|      1

24/04/12 16:11:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , masked_sent, original_phrase, suggested_completion, is_valid, is_fact, is_more_abstract, is_factually_correct, method
 Schema: _c0, masked_sent, original_phrase, suggested_completion, is_valid, is_fact, is_more_abstract, is_factually_correct, method
Expected: _c0 but found: 
CSV file: s3a://multi-token-completion/human_annotated/labeled_completions.csv


In [3]:
health_camp_deets = spark.read.format("csv").option("inferSchema", "true").option("header", "true").option("path", "./data/Health_Camp_Detail.csv").load()

In [4]:
health_camp_deets.printSchema()

root
 |-- Health_Camp_ID: integer (nullable = true)
 |-- Camp_Start_Date: string (nullable = true)
 |-- Camp_End_Date: string (nullable = true)
 |-- Category1: string (nullable = true)
 |-- Category2: string (nullable = true)
 |-- Category3: integer (nullable = true)



In [19]:
health_camp_deets.select("Health_Camp_ID", "Camp_Start_date").show(10)

+--------------+---------------+
|Health_Camp_ID|Camp_Start_date|
+--------------+---------------+
|          6560|      16-Aug-03|
|          6530|      16-Aug-03|
|          6544|      03-Nov-03|
|          6585|      22-Nov-03|
|          6561|      30-Nov-03|
|          6581|      07-Dec-03|
|          6564|      17-Dec-03|
|          6557|      04-Jan-04|
|          6538|      09-Jan-04|
|          6546|      09-Jan-04|
+--------------+---------------+
only showing top 10 rows



In [16]:
health_camp_deets.createOrReplaceTempView("health_camp_deets")

In [18]:
spark.sql("select * from health_camp_deets").show(10)

+--------------+---------------+-------------+---------+---------+---------+
|Health_Camp_ID|Camp_Start_Date|Camp_End_Date|Category1|Category2|Category3|
+--------------+---------------+-------------+---------+---------+---------+
|          6560|      16-Aug-03|    20-Aug-03|    First|        B|        2|
|          6530|      16-Aug-03|    28-Oct-03|    First|        C|        2|
|          6544|      03-Nov-03|    15-Nov-03|    First|        F|        1|
|          6585|      22-Nov-03|    05-Dec-03|    First|        E|        2|
|          6561|      30-Nov-03|    18-Dec-03|    First|        E|        1|
|          6581|      07-Dec-03|    13-Jun-04|    First|        F|        2|
|          6564|      17-Dec-03|    11-Jun-04|    First|        C|        2|
|          6557|      04-Jan-04|    09-Jan-04|    First|        C|        2|
|          6538|      09-Jan-04|    04-Feb-05|    First|        F|        2|
|          6546|      09-Jan-04|    17-Jan-04|    First|        E|        2|